This is best thought of as a "appendix" to loan_ml. I am trying to investigate the usefulness of a logistic regression (logit) since it seems to have abysmal to no predictive value.

Here I will try to find if I improve the logit function through a combination of preprocessing and hyper parameter tuning.

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('lending_club_ml.csv')
df.drop(['id'], inplace=True, axis=1)
df.info()

C:\Users\leero\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (30) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2506738 entries, 0 to 2506737
Data columns (total 100 columns):
loan_amnt                              float64
term                                   int64
int_rate                               float64
installment                            float64
grade                                  object
sub_grade                              object
emp_length                             int64
home_ownership                         object
annual_inc                             float64
verification_status                    object
loan_status                            int64
purpose                                object
dti                                    float64
delinq_2yrs                            float64
fico_range_low                         float64
fico_range_high                        float64
inq_last_6mths                         float64
mths_since_last_delinq                 float64
mths_since_last_record                 float64
open

This is basically the same code from loan_ml

In [3]:
#if a loan was an indivdual loan then we will fill the all joint-realted features with its self

df.application_type = df.application_type.apply(
    lambda app_type: 1 if app_type == 'Individual' else 0)

sec_list = ['sec_app_fico_range_low', 'sec_app_fico_range_high', 'sec_app_inq_last_6mths', 'sec_app_mort_acc',
            'sec_app_open_acc', 'sec_app_revol_util', 'sec_app_num_rev_accts', 'sec_app_chargeoff_within_12_mths',
            'sec_app_collections_12_mths_ex_med', 'sec_app_mths_since_last_major_derog', 'sec_app_open_act_il']
joint_list = ['dti_joint', 'revol_bal_joint', 'annual_inc_joint']

for secondary in sec_list:
    df.loc[df.application_type ==
           1][secondary] = df.loc[df.application_type == 1][secondary[8:]]

for joint in joint_list:
    df.loc[df.application_type ==
           1][joint] = df.loc[df.application_type == 1][joint[:len(joint)-6]]

C:\Users\leero\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
C:\Users\leero\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [4]:
#Dummy variables for catergorical features

df = pd.concat([df, pd.get_dummies(
    df.sub_grade, prefix='sub_grade', drop_first=True)], axis=1)
df = pd.concat([df, pd.get_dummies(df.home_ownership,
                                   prefix='home_ownership', drop_first=True)], axis=1)
df = pd.concat([df, pd.get_dummies(df.verification_status,
                                   prefix='verification_status', drop_first=True)], axis=1)
df = pd.concat([df, pd.get_dummies(df.purpose, prefix='purpose', drop_first=True)], axis=1)
df = pd.concat([df, pd.get_dummies(df.verification_status_joint,
                                   prefix='verification_status_joint', drop_first=True)], axis=1)

df.disbursement_method = df.disbursement_method.apply(
    lambda disburstment: 1 if disburstment == 'Cash' else 0)
df.application_type = df.application_type.apply(
    lambda app_type: 1 if app_type == 'Individual' else 0)

df.drop(columns=['sub_grade', 'home_ownership', 'verification_status',
                 'purpose', 'verification_status_joint'], inplace=True)

In [6]:
#fill nans with the average

df_avg=df

for thing in df_avg:
    if df_avg[thing].isnull().values.any():
        df_avg[thing].fillna(np.mean(df_avg[thing]), inplace=True)

In [7]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score

def scoring(clf, x, y):
    print('score: ', '{:>10}'.format(str(clf.score(x, y))))
    
    #score adjusted for loan amount
    print('balanced_accuracy_score: ', '{:>10}'.format(
        balanced_accuracy_score(y, clf.predict(x), sample_weight=x['loan_amnt'])))

    print(confusion_matrix(y, clf.predict(x)))
    print('F1 score: ', '{:>10}'.format(
        str(f1_score(y, clf.predict(x)))))
    print('precision_score: ', '{:>10}'.format(
        str(precision_score(y, clf.predict(x)))))
    
    #score adjusted for loan amount
    print('average_precision_score: ', '{:>10}'.format(average_precision_score(
        y, clf.predict(x), average='weighted', sample_weight=x['loan_amnt'])))
    print('recall_score: ', '{:>10}'.format(
        str(recall_score(y, clf.predict(x)))))
    clf.predict_proba(x)
    
    #score adjusted for loan amount
    print('roc: ', '{:>10}'.format(
        str(roc_auc_score(y, clf.predict_proba(x)[:, 1], average='weighted', sample_weight=x['loan_amnt']))))

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split


X_avg = df_avg.drop(['loan_status','grade'], axis=1)
y_avg = df_avg.loan_status

X_avg_train, X_avg_test, y_avg_train, y_avg_test = train_test_split(
X_avg, y_avg, test_size=0.3, random_state=42)

logit = LogisticRegression()
logit.fit(X_avg_train, y_avg_train)
print('LOGISTIC REGRESSION')
scoring(logit, X_avg_test, y_avg_test)

C:\Users\leero\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\leero\Anaconda3\lib\site-packages\sklearn\svm\base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LOGISTIC REGRESSION
score:  0.8009446532149326
balanced_accuracy_score:  0.5002377383591609
[[    56 149621]
 [    73 602272]]
F1 score:  0.8894625612336975
precision_score:  0.8010075901757298
average_precision_score:  0.7856673290807463
recall_score:  0.9998788069959906
roc:  0.6443311934446472


In [9]:
#While the matrix grid clearly shows that there is no charged off predictions, this is a sainty check
np.sum(df.loan_status)/len(df)

0.8013545891114269

In [12]:
#trying with nans filled with 0

df_0 = df.fillna(0)

X_0 = df_0.drop(['loan_status','grade'], axis=1)
y_0 = df_0.loan_status

X_0_train, X_0_test, y_0_train, y_0_test = train_test_split(
X_0, y_0, test_size=0.3, random_state=42)

logit = LogisticRegression()
logit.fit(X_0_train, y_0_train)
print('LOGISTIC REGRESSION')
scoring(logit, X_0_test, y_0_test)

C:\Users\leero\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\leero\Anaconda3\lib\site-packages\sklearn\svm\base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LOGISTIC REGRESSION
score:  0.8009446532149326
balanced_accuracy_score:  0.5002377383591609
[[    56 149621]
 [    73 602272]]
F1 score:  0.8894625612336975
precision_score:  0.8010075901757298
average_precision_score:  0.7856673290807463
recall_score:  0.9998788069959906
roc:  0.6443311934446472


Did not even matter, will try normalizing then hyper tuning 'C'

In [ ]:
from sklearn.preprocessing import Normalizer

trans_norm = Normalizer()

for column in df:
    if (thing.dtypes == 'float64' | thing.dtypes=='int64'):
        print(thing)



In [ ]:
import sklearn.metrics 

scorefunc = sklearn.metrics.roc_auc_score  # Replace with custom
myscorer = sklearn.metrics.make_scorer(
         scorefunc,
         greater_is_better=True,
         needs_threshold=False # ... classification
)

LogisticRegressionCV(... scoring=myscorer,)

In [ ]:
from sklearn.linear_model import LogisticRegressionCV

clf = LogisticRegressionCV(cv=5,solver='liblinear', Cs=np.logspace(np.log10(.0001),np.log10(10000),5),scoring='roc_auc_score')

clf.fit(X_avg,y_avg)
print(clf.scores_)

clf.fit(X_0,y_0)
print(clf.scores_)

C:\Users\leero\Anaconda3\lib\site-packages\sklearn\svm\base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
